In [2]:
import numpy as np
import os
import tensorflow as tf

In [3]:
feat_size = 90
context = 5

input_x = tf.placeholder(dtype = tf.float32, shape = [None, feat_size*(2*context +1)])
output_y = tf.placeholder(dtype = tf.float32, shape = [None, 2])


In [4]:
#hidden layer 1

hidden_1 = 512
hidden_1_weights = tf.get_variable( "hidden_1_weights",
                                  shape = [ feat_size*(2*context +1), hidden_1],
                                  dtype = tf.float32,
                                  initializer = tf.contrib.layers.xavier_initializer())

hidden_1_bias = tf.get_variable( "hidden_1_bias",
                                    shape = [hidden_1],
                                    dtype = tf.float32,
                                   initializer = tf.contrib.layers.xavier_initializer())

hidden_1_dnn = tf.matmul( input_x, hidden_1_weights) + hidden_1_bias
hidden_1_out = tf.nn.relu(hidden_1_dnn)
print(hidden_1_out)

Tensor("Relu:0", shape=(?, 512), dtype=float32)


In [5]:
#hidden layer 2

hidden_2 = 512
hidden_2_weights = tf.get_variable( "hidden_2_weights",
                                  shape = [ hidden_1, hidden_2],
                                  dtype = tf.float32,
                                  initializer = tf.contrib.layers.xavier_initializer())

hidden_2_bias = tf.get_variable( "hidden_2_bias",
                                    shape = [hidden_2],
                                    dtype = tf.float32,
                                   initializer = tf.contrib.layers.xavier_initializer())

hidden_2_dnn = tf.matmul( hidden_1_out, hidden_2_weights) + hidden_2_bias
hidden_2_out = tf.nn.relu(hidden_2_dnn)
print(hidden_2_out)


Tensor("Relu_1:0", shape=(?, 512), dtype=float32)


In [6]:
#hidden layer 3

hidden_3 = 512
hidden_3_weights = tf.get_variable( "hidden_3_weights",
                                  shape = [ hidden_2, hidden_3],
                                  dtype = tf.float32,
                                  initializer = tf.contrib.layers.xavier_initializer())

hidden_3_bias = tf.get_variable( "hidden_3_bias",
                                    shape = [hidden_3],
                                    dtype = tf.float32,
                                 initializer = tf.contrib.layers.xavier_initializer())

hidden_3_dnn = tf.matmul( hidden_2_out, hidden_3_weights) + hidden_3_bias
hidden_3_out = tf.nn.relu(hidden_3_dnn)
print(hidden_3_out)

Tensor("Relu_2:0", shape=(?, 512), dtype=float32)


In [7]:
#hidden layer 4

hidden_4 = 256
hidden_4_weights = tf.get_variable( "hidden_4_weights",
                                  shape = [ hidden_3, hidden_4],
                                  dtype = tf.float32,
                                  initializer = tf.contrib.layers.xavier_initializer())

hidden_4_bias = tf.get_variable( "hidden_4_bias",
                                    shape = [hidden_4],
                                    dtype = tf.float32,
                                   initializer = tf.contrib.layers.xavier_initializer())

hidden_4_dnn = tf.matmul( hidden_3_out, hidden_4_weights) + hidden_4_bias
hidden_4_out = tf.nn.relu(hidden_4_dnn)
print(hidden_4_out)


Tensor("Relu_3:0", shape=(?, 256), dtype=float32)


In [8]:
#hidden layer 5

hidden_5 = 256
hidden_5_weights = tf.get_variable( "hidden_5_weights",
                                  shape = [ hidden_4, hidden_5],
                                  dtype = tf.float32,
                                  initializer = tf.contrib.layers.xavier_initializer())

hidden_5_bias = tf.get_variable( "hidden_5_bias",
                                    shape = [hidden_5],
                                    dtype = tf.float32,
                                   initializer = tf.contrib.layers.xavier_initializer())

hidden_5_dnn = tf.matmul( hidden_4_out, hidden_5_weights) + hidden_5_bias
hidden_5_out = tf.nn.relu(hidden_5_dnn)
print(hidden_5_out)


Tensor("Relu_4:0", shape=(?, 256), dtype=float32)


In [9]:
#output layer 

output_dim = 2
output_layer_weights = tf.get_variable("output_layer_weights",
                                      shape = [hidden_5 ,output_dim],
                                      dtype = tf.float32,
                                      initializer = tf.contrib.layers.xavier_initializer())
output_layer_bias = tf.get_variable("output_layer_bias",
                                   shape = [output_dim],
                                   dtype = tf.float32,
                                   initializer = tf.contrib.layers.xavier_initializer())

output_layer_sum = tf.matmul(hidden_5_out, output_layer_weights) + output_layer_bias
final_output = tf.nn.softmax(output_layer_sum)
print(final_output)


Tensor("Softmax:0", shape=(?, 2), dtype=float32)


In [10]:
#losses and optimizers

cross_entropy = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(labels = output_y, logits = output_layer_sum))
optimizer = tf.train.AdamOptimizer(1e-2)
train_step = optimizer.minimize(cross_entropy)

correct_predictions = tf.equal(tf.argmax(final_output,1),tf.argmax(output_y,1))
accuracy = tf.reduce_mean(tf.cast(correct_predictions,tf.float32))

In [11]:
# initializing tensors

session = tf.InteractiveSession()

init = tf.global_variables_initializer()
saver = tf.train.Saver(max_to_keep = None)

features_path = 'C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\contextData_90D'
dev_features_path = 'C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\devcontextData_90D'
#eval_features_path = 'C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\evalcontextData'
model_path = 'C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\Models_90D'

dev_sp_path = 'C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\Dev_sp_90D'
#eval_sp_path = 'C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\Eval_sp'


In [12]:
def cal_eer(scores_true, scores_false):
    
    """ Calculate EER
        Args:
            scores_true: A score list of positive samples
            scores_false: A score list of negative samples
        Return:
            (EER, threshold)
        Note:
            Here, the threshold is set, and if the score is higher than or equal to the threshold,
            the decision is passed, and the decision is rejected as below.
    """
    min1 = min(scores_true)
    min2 = min(scores_false)
    low = min(min1, min2)
    max1 = max(scores_true)
    max2 = max(scores_false)
    high = max(max1, max2)
    FAR = -55
    FRR = -55
    mid = -100
    # Bisection find threshold
    while True:
        bef_FAR = FAR 
        bef_FRR = FRR
        bef_mid = mid
        mid = (high + low) / 2
        FRR = sum([1 for s in scores_true if s < mid]) / len(scores_true)
        FAR = sum([1 for s in scores_false if s >= mid]) / len(scores_false)
        #print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)

        if abs(FRR - FAR) < 1e-7:
            print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)
            break

        if FRR < FAR:
            low = mid
        else:
            high = mid
            
        if bef_FAR == FAR :
            if bef_FRR == FRR :
                if bef_mid == mid :
                    print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)
                    break

    EER  =  FRR
    return EER, (high + low) / 2

In [13]:
from scipy.io import savemat

def get_EER(test_path, index, sp_path ):
    scores = []
    development_labels = []
    probabilities = []

    #test_path = "C:\\Users\\project1\\Desktop\\Siva\\HT_IFCC_hemlata\\CNN_input_testing_dev"
    indexs = (len(list(os.listdir(test_path))))//2
    
    for folders in range(1, indexs+1):
        features = np.load(os.path.join(test_path, "features"+str(folders)+".npy"))
        labels = np.load(os.path.join(test_path, "labels"+str(folders)+".npy"))
        patches = labels.shape[0]
        patch_prob = session.run(final_output, feed_dict = {input_x:features})
        final_prob = np.zeros([2])
        
        for i in range(patches):
            if(patch_prob[i][0] == 0):
                patch_prob[i][0] = 1e-25
            if(patch_prob[i][1] == 0):
                patch_prob[i][1] = 1e-25
                
            final_prob[0] = final_prob[0] + np.log(patch_prob[i][0])
            final_prob[1] = final_prob[1] + np.log(patch_prob[i][1])

        final_prob = final_prob / patches
        score = (final_prob[0]) - (final_prob[1])
        audio_label = labels[0]

        scores.append(score)
        probabilities.append( np.mean(patch_prob, axis=0) )
        development_labels.append( audio_label )

    scores = np.array(scores)
    development_labels = np.array(development_labels)
    probabilities = np.array(probabilities)

    scores_geniue = []
    scores_spoof = []
    prob_genuine = []
    prob_spoof = []

    for i in range(development_labels.shape[0]):
        if development_labels[i][0] == 1 :
            scores_geniue.append(scores[i])
            prob_genuine.append(probabilities[i])
        elif development_labels[i][1] == 1:
            scores_spoof.append(scores[i])
            prob_spoof.append(probabilities[i])

    scores_geniue = np.array(scores_geniue)
    scores_spoof = np.array(scores_spoof)
    prob_genuine = np.array(prob_genuine)
    prob_spoof = np.array(prob_spoof)
    
    a,b = cal_eer(scores_geniue, scores_spoof)
    
    score_path = os.path.join(sp_path, "scores")
    prob_path = os.path.join(sp_path, "probabilities")
    
    if not os.path.exists(score_path):
        os.makedirs(score_path)
        
    if not os.path.exists(prob_path):
        os.makedirs(prob_path)
        
    prob_file = "probabilities_"+str(index)
    sc_file = "scores_"+str(index)
    savemat(os.path.join(score_path, sc_file), { 'genuine':scores_geniue, 'spoof':scores_spoof, 'eer' : a*100 })
    savemat(os.path.join(prob_path, prob_file), { 'genuine':prob_genuine, 'spoof':prob_spoof })
    
    return a

In [14]:
#Training and testing accuracy
session.run(init)

if not os.path.exists(model_path):
    os.makedirs(model_path)
    
if not os.path.exists(dev_sp_path):
    os.makedirs(dev_sp_path)
    
batches = (len(list(os.listdir(features_path))))//2
    
for i in range(0,50):
    for j in range(1,batches+1):  #number of batches
        filename = "batch_"+str(j)+".npy"
        labelsname= "label_"+str(j)+".npy"
        
        features = np.load(os.path.join(features_path, filename))
        labels = np.load(os.path.join(features_path, labelsname))
        
        #if(np.sum(np.isnan(features)) > 0):
        #    print(np.sum(np.isnan(features)))
        
        train_step.run(session = session,feed_dict={ input_x:features, output_y:labels})
        
    save_path = saver.save(session,os.path.join(model_path,"Model"+str(i+1)))
    
    print("iteration  ",(i+1))
    eer_dev = get_EER(dev_features_path, (i+1), dev_sp_path)
    print(" develop eer  ",eer_dev)
    print()

INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D\Model1 is not in all_model_checkpoint_paths. Manually adding it.
iteration   1
FRR   0.15921052631578947  FAR    0.16    mid    0.439770764611
 develop eer   0.15921052631578947

INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D\Model2 is not in all_model_checkpoint_paths. Manually adding it.
iteration   2
FRR   0.15921052631578947  FAR    0.15894736842105264    mid    0.84233767562
 develop eer   0.15921052631578947

INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D\Model3 is not in all_model_checkpoint_paths. Manually adding it.
iteration   3
FRR   0.14210526315789473  FAR    0.14210526315789473    mid    0.428668602799
 develop eer   0.14210526315789473

INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D\Model4 is not in all_model_checkpoint_paths. Manually adding it.
iteration   4
FRR   0.16052631578

iteration   30
FRR   0.2565789473684211  FAR    0.25684210526315787    mid    0.138122451993
 develop eer   0.2565789473684211

INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D\Model31 is not in all_model_checkpoint_paths. Manually adding it.
iteration   31
FRR   0.2  FAR    0.20105263157894737    mid    0.150610314101
 develop eer   0.2

INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D\Model32 is not in all_model_checkpoint_paths. Manually adding it.
iteration   32
FRR   0.2513157894736842  FAR    0.2536842105263158    mid    0.159382888528
 develop eer   0.2513157894736842

INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC_CMVN\Models_90D\Model33 is not in all_model_checkpoint_paths. Manually adding it.
iteration   33
FRR   0.18947368421052632  FAR    0.18947368421052632    mid    0.131788776078
 develop eer   0.18947368421052632

INFO:tensorflow:C:\Users\HIMANSHU\Desktop\Aditya\ASVspoof2017_CMPOC

In [ ]:
h1 = session.run(hidden_layer1_weights)

In [ ]:
np.sum(np.isnan(h1))

In [ ]:
eer_dev = get_EER(dev_features_path, (i+1), dev_sp_path)
print(" develop eer  ",eer_dev)
print()